In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/kansita.csv')

In [3]:
len(df)

33

In [4]:
df.head()

,name,price,energy,protein,fat,carbohydrates,salt,calcium,vegetable,img_url
0,ローストンカツおろしソース,308.0,413.0,15.6,26.0,27.0,2.4,29.0,49.0,https://west2-univ.jp/menu_img/png_sp/814022.png
1,チキンカツ柚子胡椒マヨ,264.0,406.0,16.8,27.9,19.7,1.2,25.0,33.0,https://west2-univ.jp/menu_img/png_sp/814082.png
2,ハンバーグトマトソース,264.0,228.0,15.3,11.2,15.5,1.6,52.0,70.0,https://west2-univ.jp/menu_img/png_sp/814037.png
3,豚ブロッコリー和風炒め,264.0,280.0,18.1,18.6,9.6,1.7,85.0,200.0,https://west2-univ.jp/menu_img/png_sp/814202.png
4,ビーフシチュー,264.0,316.0,9.2,16.2,30.6,1.7,22.0,80.0,https://west2-univ.jp/menu_img/png_sp/814126.png


In [5]:
#　必要量を入れた辞書
r_dic = {
    'energy': 2650 / 3,
    'protein': 65 / 3 ,
    'fat': 60 / 3,
    'carbohydrates': 300 / 3,
    'salt' : 7.5 / 3, #以下
    'calcium' : 800 / 3,
    'vegetable': 350 / 3 # 以上    
}

決定変数  
$x_{s}$ : メニューsを選ぶかどうか

In [6]:
import pulp 

In [7]:
# df['name'].to_dict()

In [8]:
S = list(range(len(df)))

In [9]:
# 上限金額
max_price = 1000
# 食べたい商品
T_plus = []
T_minus = []

In [10]:
# 問題設定
problem = pulp.LpProblem('Problem Name', pulp.LpMinimize) 
xs = pulp.LpVariable.dicts('x', S, 0, 1, 'Integer')
# スラック変数の定義
nutrition_list = ['energy', 'protein', 'fat', 'carbohydrates', 'salt', 'calcium', 'vegetable']
zs = pulp.LpVariable.dict('z', nutrition_list, lowBound=0,cat='Continuous')

/usr/local/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
zs

{'energy': z_energy,
 'protein': z_protein,
 'fat': z_fat,
 'carbohydrates': z_carbohydrates,
 'salt': z_salt,
 'calcium': z_calcium,
 'vegetable': z_vegetable}

In [12]:
# 目的関数　slack変数z
problem += pulp.lpSum(zs)

# 制約式
# 好き嫌い制約
for idx in T_plus:
    problem += xs[idx] == 1
for idx in T_minus:
    problem += xs[idx] == 0

# 金額制約
problem += pulp.lpSum(df['price'][i]*xs[s] for i,s in enumerate(S)) <= max_price

# 栄養制約
for nut in nutrition_list:    
    if nut == 'salt':
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] <= zs[nut]*r_dic[nut]
    if nut == 'vegetable':
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] >= -zs[nut]*r_dic[nut]
    else :
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] <= zs[nut]*r_dic[nut]
        problem += pulp.lpSum(df[nut][i]*xs[s] for i,s in enumerate(S)) - r_dic[nut] >= -zs[nut]*r_dic[nut]

In [13]:
sol = problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/1891a79f1d3d493e9a06887c5338899c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/kc/5vks2chj5jg_zq9fvglb01jr0000gp/T/1891a79f1d3d493e9a06887c5338899c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 586 RHS
At line 602 BOUNDS
At line 636 ENDATA
Problem MODEL has 15 rows, 40 columns and 492 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.230343 - 0.00 seconds
Cgl0004I processed model has 14 rows, 40 columns (33 integer (33 of which binary)) and 460 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.36329
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 0.891715 iterations 13
Cbc0038I Solution found of 0.891715

In [14]:
pulp.LpStatus[sol]

'Optimal'

In [15]:
ret_dic = {}
total_price = 0
for nut in nutrition_list:
    ret_dic[nut] = 0.
for i in xs:
    if xs[i].value():
        total_price += df['price'][i]
        for nut in nutrition_list:
            ret_dic[nut] += df[nut][i]

In [16]:
for nut in nutrition_list:
    print(nut, ret_dic[nut] - r_dic[nut])

energy -294.33333333333337
protein 1.9333333333333336
fat -0.8000000000000007
carbohydrates -16.900000000000006
salt 0.10000000000000009
calcium 0.3333333333333144
vegetable 57.33333333333333


In [17]:
for i in zs:
    print(zs[i].value())

0.33320755
0.089230769
0.04
0.169
0.04
0.00125
0.0


In [18]:
print(f"目的関数 : {problem.objective.value()}")
print(f"価格 : {total_price}　円")
for i in range(len(S)):
    if xs[i].value():
        print(df['name'][i], df['price'][i])

目的関数 : 0.672688319
価格 : 579.0　円
ポテト野菜サラダ 110.0
蒸し鶏ビーンズサラダ 176.0
豚汁 110.0
大学芋 88.0
大学生協コーヒー 95.0


In [19]:
df.iloc[[i for i,v in enumerate(xs.values()) if v.value() == 1]]

,name,price,energy,protein,fat,carbohydrates,salt,calcium,vegetable,img_url
8,ポテト野菜サラダ,110.0,98.0,1.7,5.2,11.9,0.5,17.0,53.0,https://west2-univ.jp/menu_img/png_sp/814418.png
16,蒸し鶏ビーンズサラダ,176.0,91.0,10.5,1.9,9.1,0.3,42.0,67.0,https://west2-univ.jp/menu_img/png_sp/814325.png
17,豚汁,110.0,102.0,6.0,3.8,11.2,1.7,33.0,54.0,https://west2-univ.jp/menu_img/png_sp/814400.png
29,大学芋,88.0,156.0,0.6,3.1,31.9,0.0,13.0,0.0,https://west2-univ.jp/menu_img/png_sp/814817.png
32,大学生協コーヒー,95.0,142.0,4.8,5.2,19.0,0.1,162.0,0.0,https://west2-univ.jp/menu_img/png_sp/814818.png
